In [ ]:
!pip install -q -U kaggle_environments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Опишем поведение агентов, всего их будет 12. У каждого из них своя стратегия игры.
Для удобства будем считать, что:
0 - камень
1 - бумага
2 - ножницы

Агент всегда выбирает камень.

In [ ]:
%%writefile rock_agent.py
def rock_agent(observation, configuration):
    return 0


Writing rock_agent.py


Агент всегда выбирает бумагу.

In [ ]:
%%writefile paper_agent.py
def paper_agent(observation, configuration):
    return 1


Writing paper_agent.py


Агент всегда выбирает ножницы.

In [ ]:
%%writefile scissors_agent.py
def scissors_agent(observation, configuration):
    return 2


Writing scissors_agent.py


Агент копирует последний ход противника.

In [ ]:
%%writefile copy_opponent.py
import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)


Writing copy_opponent.py


Агент выбирает случайный ход в каждом раунде

In [ ]:
%%writefile random_agent.py
import random

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)


Writing random_agent.py


Агент в первом раунде выбирает камень, а в последующих раундах повторяет ход противника.

In [ ]:
%%writefile mixed_strategy_agent.py
def mixed_strategy_agent(observation, configuration):
    if observation.step == 0:
        return 0
    else:
        return observation.lastOpponentAction


Writing mixed_strategy_agent.py


Агент выбирает камень, если проиграл в предыдущем раунде, иначе выбирает случайный ход.


In [ ]:
%%writefile rock_after_loss_agent.py
import random

def rock_after_loss_agent(observation, configuration):
    if observation.step > 0 and observation.lastOpponentAction == (observation.lastAction + 1) % 3:
        return 0
    else:
        return random.randrange(0, configuration.signs)


Writing rock_after_loss_agent.py


Агент выбирает бумагу, если он выиграл в предыдущем раунде, иначе выбирает случайный ход.


In [ ]:
%%writefile paper_after_win_agent.py
import random

def paper_after_win_agent(observation, configuration):
    if observation.step > 0 and observation.lastOpponentAction == (observation.lastAction - 1) % 3:
        return 1
    else:
        return random.randrange(0, configuration.signs)


Writing paper_after_win_agent.py


Агент поочередно выбирает камень и бумагу (начинает с камня) в каждом раунде.

In [ ]:
%%writefile alternating_choice_agent.py
import random

def alternating_choice_agent(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    elif observation.step % 2 == 0:
        return 0
    else:
        return 1


Writing alternating_choice_agent.py


Агент всегда выбирает тот ход, который победит ход противника в предыдущем раунде.

In [ ]:
%%writefile winning_choice_agent.py
import random

def winning_choice_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % 3  # Побеждает ход противника
    else:
        return random.randrange(0, configuration.signs)


Writing winning_choice_agent.py


Агент всегда выбирает противоположный ход по отношению к последнему ходу противника.

In [ ]:
%%writefile opposite_choice_agent.py
import random

def opposite_choice_agent(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    else:
        return (observation.lastOpponentAction + 1) % 3  # Противоположный ход


Writing opposite_choice_agent.py


Агент всегда выбирает следующий ход в цикле камень → бумага → ножницы.

In [ ]:
%%writefile cycle_choice_agent.py
def cycle_choice_agent(observation, configuration):
    if observation.step == 0:
        return 0  # Камень
    else:
        return (observation.lastOpponentAction + 1) % 3  # Циклично меняет ход


Writing cycle_choice_agent.py


Проведем турнир при помощи функции evaluate

In [ ]:
from kaggle_environments import evaluate

# Список из разных агентов для корректной работы функции evaluate
agents = [
    "rock_agent.py",
    "paper_agent.py",
    "scissors_agent.py",
    "copy_opponent.py",
    "random_agent.py",
    "mixed_strategy_agent.py",
    "rock_after_loss_agent.py",
    "paper_after_win_agent.py",
    "alternating_choice_agent.py",
    "winning_choice_agent.py",
    "opposite_choice_agent.py",
    "cycle_choice_agent.py"
]

# Турнир
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        print(f"Турнир между {agents[i]} и {agents[j]}:")
        results = evaluate(
            "rps",  # Окружение
            [agents[i], agents[j]],  # Агенты
            configuration={"episodeSteps": 100}  # Количество шагов
        )
        print(f"Результаты: {results}")


Турнир между rock_agent.py и paper_agent.py:
Результаты: [[-99.0, 99.0]]
Турнир между rock_agent.py и scissors_agent.py:
Результаты: [[99.0, -99.0]]
Турнир между rock_agent.py и copy_opponent.py:
Результаты: [[0, 0]]
Турнир между rock_agent.py и random_agent.py:
Результаты: [[0, 0]]
Турнир между rock_agent.py и mixed_strategy_agent.py:
Результаты: [[0, 0]]
Турнир между rock_agent.py и rock_after_loss_agent.py:
Результаты: [[1, None]]
Турнир между rock_agent.py и paper_after_win_agent.py:
Результаты: [[1, None]]
Турнир между rock_agent.py и alternating_choice_agent.py:
Результаты: [[-49.0, 49.0]]
Турнир между rock_agent.py и winning_choice_agent.py:
Результаты: [[-98.0, 98.0]]
Турнир между rock_agent.py и opposite_choice_agent.py:
Результаты: [[-97.0, 97.0]]
Турнир между rock_agent.py и cycle_choice_agent.py:
Результаты: [[-98.0, 98.0]]
Турнир между paper_agent.py и scissors_agent.py:
Результаты: [[-99.0, 99.0]]
Турнир между paper_agent.py и copy_opponent.py:
Результаты: [[0, 0]]
Турнир